# SmartFigure context-managed parameters (manual demo)

Automatable semantics are covered in `tests/test_Figure_module_params.py` and `tests/test_param_hook_context_binding.py`.
This notebook remains for interactive/context UX demonstration.


In [ ]:
import sys
from pathlib import Path

import sympy as sp

ROOT = Path.cwd().resolve().parents[1]
sys.path.insert(0, str(ROOT))
from gu_toolkit import Figure, params

In [ ]:
x, a = sp.symbols("x a")
fig = Figure(x_range=(-6, 6), y_range=(-3, 3))
with fig:
    fig.plot(a * sp.sin(x), x, id="a_sin")
    params[a].value = 2
fig

## Manual checklist
- [ ] Curve updates when parameter value is changed.
- [ ] Parameter edits through `params[...]` are reflected in the widget.
